In [ ]:
install.packages("ISLR2")
install.packages("glmnet")
install.packages("keras")
install.packages("lubridate")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘iterators’, ‘foreach’, ‘shape’, ‘Rcpp’, ‘RcppEigen’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘RcppTOML’, ‘here’, ‘png’, ‘config’, ‘tfautograph’, ‘reticulate’, ‘tensorflow’, ‘tfruns’, ‘zeallot’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
library(ISLR2)
library(glmnet)
library(keras)
library(lubridate)

In [ ]:
NYSE = read.csv('night_for_R.csv',header=FALSE)

In [ ]:
NYSE[,3] = TRUE
NYSE[c(7001:10000),3] = FALSE

In [ ]:
colnames(NYSE) = c('V1','V2','train')
colnames(NYSE)

[1] "V1"    "V2"    "train"

In [ ]:
#NYSE

In [ ]:
lagm <- function(x, k = 1) {
  n <- nrow(x)
  pad <- matrix(NA, k, ncol(x))
  rbind(pad, x[1:(n - k), ])
}

In [ ]:
xdata <- data.matrix(
 NYSE[, c("V1","V2")]
 )
istrain <- NYSE[, "train"]
xdata <- scale(xdata)
xdata = as.data.frame(xdata)

In [ ]:
arframe <- data.frame(log_volume = xdata[, "V2"],
   L1 = lagm(xdata, 1), L2 = lagm(xdata, 2),
   L3 = lagm(xdata, 3), L4 = lagm(xdata, 4),
   L5 = lagm(xdata, 5)
 )
arframe <- arframe[-(1:5), ]
istrain <- istrain[-(1:5)]
arframe = arframe[,c(1, 3, 5, 7, 9, 11)]

In [ ]:
n <- nrow(arframe)
xrnn <- data.matrix(arframe[, -2])
xrnn <- array(xrnn, c(n, 1, 5))
xrnn <- xrnn[,, 5:1]
#xrnn <- aperm(xrnn, c(1, 3, 2))
dim(xrnn)

[1] 9995    5

In [ ]:
model <- keras_model_sequential() %>%
   layer_simple_rnn(units = 12,
      input_shape = list(5, 1),
      dropout = 0.1, recurrent_dropout = 0.1) %>%
   layer_dense(units = 1)
model %>% compile(optimizer = optimizer_rmsprop(),
    loss = "mse")
history <- model %>% fit(
    xrnn[istrain,, ], arframe[istrain, "log_volume"],
    batch_size = 100, epochs = 5,
    validation_data =
      list(xrnn[!istrain,, ], arframe[!istrain, "log_volume"])
  )

ERROR: ignored

In [ ]:
npred <- predict(model, xrnn[!istrain,, ])
1 - sum((kpred - arframe[!istrain, "log_volume"])^2)/sum((mean(arframe[!istrain, "log_volume"]) - arframe[!istrain, "log_volume"])^2)
mean((kpred - arframe[!istrain, "log_volume"])^2)

[1] NaN

[1] NaN

In [ ]:
model

Model: "sequential_10"
________________________________________________________________________________
 Layer (type)                       Output Shape                    Param #     
 simple_rnn_10 (SimpleRNN)          (None, 12)                      204         
 dense_10 (Dense)                   (None, 1)                       13          
Total params: 217
Trainable params: 217
Non-trainable params: 0
________________________________________________________________________________